In [1]:
#url_format = "https://www.nba.com/stats/teams/advanced?Season={}"
url_format = "https://www.basketball-reference.com/leagues/NBA_{}.html"

In [2]:
# Lebron was drafted in 03
'''
years = [
    "2003-04", "2004-05", "2005-06", "2006-07", "2007-08", "2008-09", 
    "2009-10", "2010-11", "2011-12", "2012-13", "2013-14", "2014-15", 
    "2015-16", "2016-17", "2017-18", "2018-19", "2019-20", "2020-21", 
    "2021-22", "2022-23", "2023-24", "2024-25"
]
'''
years = [
    "2004", "2005", "2006", "2007", "2008", "2009", 
    "2010", "2011", "2012", "2013", "2014", "2015", 
    "2016", "2017", "2018", "2019", "2020", "2021", 
    "2022", "2023", "2024", "2025"
]


In [3]:
teams = ["TOR"]

In [ ]:
%pip install selenium
# Need to download webdriver

In [ ]:
# Alternative workflow using NBA.com (Selenium may not be the best for webscraping moving forward)
'''
import os

for team in teams:
    for year in years:
        url = url_format.format(year)

        directory = f"data/{team}/{year}"
        if not os.path.exists(directory):
            os.makedirs(directory)

        from selenium.webdriver.chrome.service import Service

        service = Service(executable_path='/Users/aaronchow/Desktop/chromedriver-mac-arm64/chromedriver')
        options = webdriver.ChromeOptions()
        driver = webdriver.Chrome(service=service, options=options)

        driver.get(url)
        html = driver.page_source

        with open(f"{folder_name}/team_ratings.html", "w+") as f:
            f.write(html)
'''

In [4]:
import certifi
import urllib3
http = urllib3.PoolManager(
    cert_reqs='CERT_REQUIRED',
    ca_certs=certifi.where()
)

In [ ]:
# May want to use this instead: https://www.basketball-reference.com/leagues/NBA_2023.html
# 1 min 13.5 seconds on initial run
import os
import time
import requests
from io import StringIO
import pandas as pd
from bs4 import BeautifulSoup

dfs = []

for year in years:
    url = url_format.format(year)

    directory = f"data/advancedStats/{year}"
    if not os.path.exists(directory):
        os.makedirs(directory)

    response = requests.get(url, verify=False)
     #response = requests.get(url, verify=certifi.where())
    soup = BeautifulSoup(response.content, 'html.parser')

    advanced_stats_table = soup.find("table", {"id": "advanced-team"})

    useless_headers = soup.find_all('tr', class_="over_header")

    for header in useless_headers:
        header.decompose()

    advanced_stats = pd.read_html(StringIO(str(advanced_stats_table)))[0]

    dfs.append(advanced_stats)

    time.sleep(2)
        

In [23]:
advanced_stats = pd.concat(dfs)
advanced_stats.drop(['Arena', "Attend.", "Attend./G", "Unnamed: 17", "Unnamed: 22", "Unnamed: 27"], axis=1, inplace=True)
advanced_stats = advanced_stats[advanced_stats['Team'] != 'League Average']
advanced_stats['Team'] = advanced_stats['Team'].str.replace('*', '', regex=False)


In [24]:
advanced_stats.to_csv("advanced_stats.csv")

In [ ]:
advanced_stats

In [ ]:
'''
Concerns
    - At some point, the Charlotte Bobcats became the Hornets, and then moved to New Orleans
        - We need to account for these name differences
'''